In [1]:
!pip install confluent-kafka

In [2]:
!pip install confluent-kafka[avro]

In [3]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [4]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [5]:
conf = {'bootstrap.servers': bootstrap_server}

In [6]:
a = AdminClient(conf)

In [7]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    ##if not (str(t)).startswith("_"):
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 9 topics:
  "_kafka-connect-01-offsets" with 25 partition(s)
  "_schemas" with 1 partition(s)
  "SmokeSensorEvent" with 1 partition(s)
  "_kafka-connect-01-configs" with 1 partition(s)
  "quantia-analyticsksql_processing_log" with 1 partition(s)
  "__consumer_offsets" with 50 partition(s)
  "_kafka-connect-01-status" with 5 partition(s)
  "_confluent-ksql-quantia-analytics_command_topic" with 1 partition(s)
  "__transaction_state" with 50 partition(s)


In [8]:
#fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

#for topic, f in fs.items():
#    try:
#        f.result()  # The result itself is None
#        print("Topic {} created".format(topic))
#    except Exception as e:
#        print("Failed to create topic {}: {}".format(topic, e))

## Define the value schema

In [9]:
value_schema_str = """
{
  "namespace": "continuous.analytics",
  "type": "record",
  "name": "TemperatureSensorEvent",
  "fields": [
    {
      "name": "sensor",
      "type": "string"
    },
    {
      "name": "temperature",
      "type": "float"
    },
    {
      "name": "ts",
      "type": "long"
    }
  ]
}
"""

In [10]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
import time

topic = "TemperatureSensorEvent"
schema_registry_url = "http://schema-registry:8081"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [11]:
schema_registry_conf = {'url': schema_registry_url}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [12]:
avro_serializer = AvroSerializer(value_schema_str, schema_registry_client)

producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': avro_serializer
}

producer = SerializingProducer(producer_conf)

## run the following cell to demostrate that fire is not detected

In [13]:
from random import gauss

while True:
    key = "S1"
    value = {"sensor": "S1","temperature": gauss(20, 1.0),"ts":int(time.time()*1000.0)}
    producer.produce(topic=topic, value=value, key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)


{'sensor': 'S1', 'temperature': 50.338292206306306, 'ts': 1605623936394}
{'sensor': 'S1', 'temperature': 50.70999515318572, 'ts': 1605623947413}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 0
{'sensor': 'S1', 'temperature': 51.37361878403081, 'ts': 1605623957422}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 1
{'sensor': 'S1', 'temperature': 48.77852041692449, 'ts': 1605623967432}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 2
{'sensor': 'S1', 'temperature': 50.641243977894305, 'ts': 1605623977442}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 3
{'sensor': 'S1', 'temperature': 50.502307487199815, 'ts': 1605623987447}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 4
{'sensor': 'S1', 'temperature': 53.205666723532524, 'ts': 1605623997458}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 5
{'sensor': 'S1', 'temperature': 50.86978686788763, 'ts

KeyboardInterrupt: 

## run the following cell to demostrate to detect fire

In [ ]:
while True:
    key = "S1"
    value = {"sensor": "S1","temperature": gauss(55, 1.0),"ts":int(time.time()*1000.0)}
    producer.produce(topic=topic, value=value, key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)

{'sensor': 'S1', 'temperature': 57.08832133787175, 'ts': 1605624545856}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 58
{'sensor': 'S1', 'temperature': 55.31187192662198, 'ts': 1605624555859}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 59
{'sensor': 'S1', 'temperature': 54.975452079050946, 'ts': 1605624565870}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 60
{'sensor': 'S1', 'temperature': 56.14230348444072, 'ts': 1605624575880}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 61
{'sensor': 'S1', 'temperature': 52.62000026448661, 'ts': 1605624585890}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 62
{'sensor': 'S1', 'temperature': 54.49939646612739, 'ts': 1605624595901}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 63
{'sensor': 'S1', 'temperature': 54.318403346873446, 'ts': 1605624605908}
Produced record to topic TemperatureSensorEvent pa